# EDA

In [9]:
# check that the notebook is using the correct virtual environment
# if you dont call it `umads_venv`, change the assertion

import sys
venv = sys.executable.split('/')[-3]
assert venv == 'umads_venv'